# Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from utils import create_date, drop_smart_contract, clean_up_row, drop_missing_data
from data_preparation import train_data_loader, data_pre_processing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LogisticRegression

import networkx as nx
import operator

## Load data

In [2]:
data = train_data_loader()

/home/erika/eth_fraud_detection/data_preparation.py:16: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_20170702 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_trans_20170702.csv")
/home/erika/eth_fraud_detection/data_preparation.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_20170707 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_trans_20170707.csv")
/home/erika/eth_fraud_detection/data_preparation.py:22: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_20170708 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_trans_20170708.csv")
/home/erika/eth_fraud_detection/data_preparation.py:23: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_20170709 = pd.read_csv("https://s3.eu-central-

# Data Preprocessing

In [3]:
df = data_pre_processing(data)

/home/erika/eth_fraud_detection/utils.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_drop.drop(columns=['check_hash'], inplace=True)
/home/erika/eth_fraud_detection/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_missing_date.drop(['receipt_status', 'max_fee_per_gas', 'max_priority_fee_per_gas', 'transaction_type','receipt_contract_address'],axis=1,inplace=True)
/home/erika/eth_fraud_detection/utils.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [4]:
!nvidia-smi

Sat Jul 15 13:19:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:3B:00.0 Off |                  Off |
|  0%   36C    P8               26W / 260W|     10MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Drop target features (comments, flags)
train_data = df.drop(['comment_from_address_darklst','date_mal_trans_from', 'mal_trans_from', 'comment_to_address_darklst',
                    'date_mal_trans_to', 'mal_trans_to','transaction_flag1', 'transaction_flag2','attack_descr', 'attack_date'],axis=1)


# Keep only numeric features
train_data = train_data[['nonce', 'transaction_index',
       'value', 'gas', 'gas_price', 'receipt_cumulative_gas_used',
       'receipt_gas_used', 'block_number','receipt_effective_gas_price', 
       'dates', 'gas_price_unit','value_div_gas', 'from_address_count', 'to_address_count',
       'block_count', 'degree_centrality_from', 'degree_centrality_to',
       'in_degree_adr_to', 'out_degree_adr_to', 'in_degree_adr_from','out_degree_adr_from','transaction_flag']]

In [6]:
Xtrain = train_data.drop(['transaction_flag'],axis=1)
Xtrain.set_index('dates', inplace=True)
ytrain = train_data[['transaction_flag','dates']]
ytrain.set_index('dates', inplace=True)

## Check feature and remove reduntandant features

In [7]:
Xtrain.head()

,nonce,transaction_index,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,block_number,receipt_effective_gas_price,gas_price_unit,value_div_gas,from_address_count,to_address_count,block_count,degree_centrality_from,degree_centrality_to,in_degree_adr_to,out_degree_adr_to,in_degree_adr_from,out_degree_adr_from
dates,,,,,,,,,,,,,,,,,,,,
2017-07-31,5,1,0.000000e+00,53759.0,60000000000,74758.0,53758.0,4098983.0,6.000000e+10,1.116092e+06,0.000000e+00,3,4858,171,0.000002,0.003502,4858,0,0,3
2017-07-31,2596723,172,5.459820e+17,50000.0,4000000000,5307522.0,21000.0,4101279.0,4.000000e+09,8.000000e+04,1.091964e+13,357662,32,180,0.257829,0.000046,32,32,6,357662
2017-07-31,2591374,15,2.077429e+17,50000.0,4000000000,336000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.154859e+12,357662,43,93,0.257829,0.000062,43,43,6,357662
2017-07-31,2591369,10,2.178482e+17,50000.0,4000000000,231000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.356964e+12,357662,56,93,0.257829,0.000051,56,15,6,357662
2017-07-31,2591373,14,5.256675e+16,50000.0,4000000000,315000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,1.051335e+12,357662,60,93,0.257829,0.000044,60,1,6,357662


In [8]:
ytrain.head()

,transaction_flag
dates,
2017-07-31,0
2017-07-31,0
2017-07-31,0
2017-07-31,0
2017-07-31,0


In [9]:
ytrain['transaction_flag'].value_counts()

transaction_flag
0    7457565
1       3290
Name: count, dtype: int64

## Fit the Logistic regression

In [10]:
Xtrain

,nonce,transaction_index,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,block_number,receipt_effective_gas_price,gas_price_unit,value_div_gas,from_address_count,to_address_count,block_count,degree_centrality_from,degree_centrality_to,in_degree_adr_to,out_degree_adr_to,in_degree_adr_from,out_degree_adr_from
dates,,,,,,,,,,,,,,,,,,,,
2017-07-31,5,1,0.000000e+00,53759.0,60000000000,74758.0,53758.0,4098983.0,6.000000e+10,1.116092e+06,0.000000e+00,3,4858,171,0.000002,0.003502,4858,0,0,3
2017-07-31,2596723,172,5.459820e+17,50000.0,4000000000,5307522.0,21000.0,4101279.0,4.000000e+09,8.000000e+04,1.091964e+13,357662,32,180,0.257829,0.000046,32,32,6,357662
2017-07-31,2591374,15,2.077429e+17,50000.0,4000000000,336000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.154859e+12,357662,43,93,0.257829,0.000062,43,43,6,357662
2017-07-31,2591369,10,2.178482e+17,50000.0,4000000000,231000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.356964e+12,357662,56,93,0.257829,0.000051,56,15,6,357662
2017-07-31,2591373,14,5.256675e+16,50000.0,4000000000,315000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,1.051335e+12,357662,60,93,0.257829,0.000044,60,1,6,357662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-01,3096,52,3.000000e+15,90000.0,20000000000,1941089.0,21000.0,3957068.0,2.000000e+10,2.222222e+05,3.333333e+10,1479,1,105,0.001067,0.000001,1,1,1,1479
2017-07-01,0,59,1.565260e+18,90000.0,20000000000,1385309.0,21000.0,3956044.0,2.000000e+10,2.222222e+05,1.739178e+13,3,2,101,0.000004,0.000002,2,1,2,3
2017-07-01,0,54,1.000000e+17,90000.0,20000000000,1261717.0,21000.0,3956044.0,2.000000e+10,2.222222e+05,1.111111e+12,3,2,101,0.000004,0.000002,2,1,2,3


In [11]:
Xtrain.head()

,nonce,transaction_index,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,block_number,receipt_effective_gas_price,gas_price_unit,value_div_gas,from_address_count,to_address_count,block_count,degree_centrality_from,degree_centrality_to,in_degree_adr_to,out_degree_adr_to,in_degree_adr_from,out_degree_adr_from
dates,,,,,,,,,,,,,,,,,,,,
2017-07-31,5,1,0.000000e+00,53759.0,60000000000,74758.0,53758.0,4098983.0,6.000000e+10,1.116092e+06,0.000000e+00,3,4858,171,0.000002,0.003502,4858,0,0,3
2017-07-31,2596723,172,5.459820e+17,50000.0,4000000000,5307522.0,21000.0,4101279.0,4.000000e+09,8.000000e+04,1.091964e+13,357662,32,180,0.257829,0.000046,32,32,6,357662
2017-07-31,2591374,15,2.077429e+17,50000.0,4000000000,336000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.154859e+12,357662,43,93,0.257829,0.000062,43,43,6,357662
2017-07-31,2591369,10,2.178482e+17,50000.0,4000000000,231000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,4.356964e+12,357662,56,93,0.257829,0.000051,56,15,6,357662
2017-07-31,2591373,14,5.256675e+16,50000.0,4000000000,315000.0,21000.0,4099488.0,4.000000e+09,8.000000e+04,1.051335e+12,357662,60,93,0.257829,0.000044,60,1,6,357662


In [12]:
ytrain.head()

,transaction_flag
dates,
2017-07-31,0
2017-07-31,0
2017-07-31,0
2017-07-31,0
2017-07-31,0


In [16]:
lr = LogisticRegression(random_state=0).fit(Xtrain,ytrain)
lr.predict(Xtrain)

/home/erika/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/erika/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0, 0, ..., 0, 0, 0])

In [17]:
labels = pd.Series(lr.predict(Xtrain))
pd.unique(labels)
len(pd.unique(labels))

1

In [18]:
ytrain.value_counts()

transaction_flag
0                   7457565
1                      3290
Name: count, dtype: int64

In [19]:
labels.value_counts()

0    7460855
Name: count, dtype: int64

Logistic Regression would predict all the observion to be non anomalies